In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
{ "name": [ "spam", "lang_mismatch", "pii", "not_appropriate", "hate_speech", "sexual_content", "quality", "toxicity", "humor", "creativity", "violence" ], 
 "value": [ 0, 0, 0, 0, 0, 0, 0.9166666666666666, 0.16666666666666666, 0.3333333333333333, 0.6666666666666666, 0 ], 
 "count": [ 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3 ] }

{'name': ['spam',
  'lang_mismatch',
  'pii',
  'not_appropriate',
  'hate_speech',
  'sexual_content',
  'quality',
  'toxicity',
  'humor',
  'creativity',
  'violence'],
 'value': [0,
  0,
  0,
  0,
  0,
  0,
  0.9166666666666666,
  0.16666666666666666,
  0.3333333333333333,
  0.6666666666666666,
  0],
 'count': [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]}

In [4]:


# Creativity
# examples = get_open_assistant_messages(label_key="creativity", max_messages=512)

# Quality
# examples = get_open_assistant_messages(label_key="quality", max_messages=512)

# Compliance
# WARNING: ToxicCompletions contains offensive/harmful user prompts
# examples = get_toxic_completions_messages(max_messages=512)

# Truthfulness
# examples = get_truthfulqa_messages(max_messages=512)
from transformers import AutoModelForCausalLM, AutoTokenizer
from concept_guidance.activations import compute_activations
from concept_guidance.models.difference_in_means import DiMProbe
import torch
device = torch.device("cuda:6")

model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1", device_map = device)
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")



Loading checkpoint shards: 100%|██████████| 2/2 [00:16<00:00,  8.09s/it]


In [3]:
from transformers import pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
prompt = "Q: Richard lives in an apartment building with 15 floors. Each floor contains 8 units, and 3/4 of the building is occupied. \
What's the total number of unoccupied units In the building? A:"
output = pipe(prompt, max_new_tokens=200, return_full_text=False)

print(output[0]["generated_text"])

2024-04-04 06:34:43.148550: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 105


In [4]:
from concept_guidance.data.open_assistant import get_open_assistant_messages
from concept_guidance.data.toxic_completions import get_toxic_completions_messages
# from concept_guidance.data.truthfulqa import get_truthfulqa_messages

# Humor
examples = get_open_assistant_messages(label_key="humor", max_messages=512)
print(examples)

[{'prompt': 'I have ground turkey, eggs, and various spices and vegetables.  What is your favorite turkey burger recipe?', 'completion': 'I found this recipe from AllRecipies:\n\n3 pounds Ground Turkey\n¼ cup Seasoned Bread Crumbs\n¼ cup Onion (finely diced)\n2 Egg Whites (lightly beaten)\n¼ cup Fresh Parsley (chopped)\n1 clove Garlic (peeled and minced)\n1 teaspoon Salt\n¼ teaspoon Black Pepper (ground)\n\nWould you like preparation instructions?', 'conversation_history': None, 'label': 0}, {'prompt': 'I am feeling quite sad lately..', 'completion': 'What has been going on?', 'conversation_history': None, 'label': 1}, {'prompt': 'Explain how quantum tunneling works, but in the style of cavemen speak.', 'completion': "Me already explained quantum tunneling in cavemen language! Me talk about atoms, waves, and big mountain. Remember? Quantum tunneling is when atom sometimes go through big mountain like magic trick. It happen because atom act like wave and spread out to both sides of moun

In [3]:
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
import json
with open("concept_guidance/data/generated2.json", "r") as f:
    examples = json.load(f)
print(examples)

[{'question': "Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?", 'answer': '\n\n1. How many eggs does she have left after breakfast and baking?\n\n16 - 3 - 4 = 9\n\n2. How much does she make from selling 9 eggs?\n\n9 x $2 = $18\n\n3. How much does she make every day?\n\n$18 + $18 + $18 + $18 + $18 + $18 + $18 + $18 + $18 + $18 + $18 + $18 + $18 + $18 + $18 + $18 = $18 x 16 = $288\n\nSo, she makes $288 every day.', 'label': 1}, {'question': "Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?", 'answer': "16 - 3 - 4 = 9 eggs per day.

In [4]:
from concept_guidance.activations import compute_activations
# Compute model activations
activations, labels = compute_activations(model, tokenizer, examples)



Activations:   0%|          | 0/602 [00:00<?, ?it/s]

Activations: 100%|██████████| 602/602 [02:16<00:00,  4.40it/s]


In [14]:
from concept_guidance.models.logistic import LogisticProbe
# Train a probe on the activations
probe = LogisticProbe()  # or LogisticProbe() or PCAProbe()
probe.fit(activations, labels)

# To get the vectors directly
concept_vectors = probe.get_concept_vectors()

# To save the probe
probe.save("concepttest3.safetensors")

  0%|          | 0/32 [00:00<?, ?it/s]

In [6]:
print(activations[1].shape) # (seq_len, num_layer, dim_hidden)
print(concept_vectors.shape)

torch.Size([127, 32, 4096])
torch.Size([32, 4096])


In [132]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from concept_guidance.patching import load_weights, patch_model, unpatch_model

concept_vectors = load_weights("concepttest2.safetensors")

# Patch the model with the concept vectors
# Note: the guidance scale is highly dependent on the model and concept
patch_model(model, concept_vectors, guidance_scale = 20.0, guidance_layers = range(1,32))

# Create a pipeline with the patched model
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Generate text with concept guidance
prompt = tokenizer.apply_chat_template([{"role": "user", "content": "What's it like to walk on the moon?"}], tokenize=False)
# prompt = tokenizer.apply_chat_template([{"Q": "user", "A": "Charlie wants to sell beeswax candles.  For every pound of beeswax, he can make 10 tapered candles.  One pound of beeswax and the wicks cost $10.00 in supplies.   If he sells each candle for $2.00 each, what is his net profit if he makes and sells 20 candles?"}], tokenize=False)
prompt = "question: There are 25 roses in a garden. There are 40 tulips. There are 35 daisies. What percentage of flowers are not roses?\nanswer:"
output = pipe(prompt, max_new_tokens=150, return_full_text=False)

print(output[0]["generated_text"])
# >>> Well, I've never been there, but I heard it's pretty rocky.

# Remove concept guidance
unpatch_model(model)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 65% (40 tulips + 35 daisies) / 100% (40 tulips + 35 daisies + 25 roses) = 65%


MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
    (norm): MistralRMSNorm(

In [11]:
concept_vectors.shape

torch.Size([32, 4096])